<a href="https://colab.research.google.com/github/ravindrabharathi/tf_utils/blob/active_learning_drought_watch/test/Copy_of_droughtwatch_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Concatenate,Lambda,Activation,Input,Dropout
 
from tensorflow.keras.optimizers import Adam,SGD
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.data import Dataset

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

import math

In [2]:
!pip install wandb -qqq
import wandb
wandb.login()


     |████████████████████████████████| 2.1MB 12.7MB/s 
     |████████████████████████████████| 174kB 28.7MB/s 
     |████████████████████████████████| 102kB 12.3MB/s 
     |████████████████████████████████| 133kB 48.9MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
run=wandb.init(name='draughtwatch_9', 
           project='Wandb_Drought_Watch',
           notes='Drought Watch dataset with Bands B2-B7, custome ResNet', 
           tags=['DroughtWatch', 'tf_utils','ResNet'])

wandb: Currently logged in as: ravindra (use `wandb login --relogin` to force relogin)


In [4]:
!pip install --upgrade git+https://github.com/ravindrabharathi/tf_utils@active_learning_drought_watch

  Cloning https://github.com/ravindrabharathi/tf_utils (to revision active_learning_drought_watch) to /tmp/pip-req-build-emnfg_cd
  Running command git clone -q https://github.com/ravindrabharathi/tf_utils /tmp/pip-req-build-emnfg_cd
  Running command git checkout -b active_learning_drought_watch --track origin/active_learning_drought_watch
  Switched to a new branch 'active_learning_drought_watch'
  Branch 'active_learning_drought_watch' set up to track remote branch 'active_learning_drought_watch' from 'origin'.
  Created wheel for tf-utils: filename=tf_utils-0.4-cp37-none-any.whl size=8986 sha256=65578ee97ec492cddbcf3720c67779a924ab81547dfbf6d2d95943e8c23b0c68
  Stored in directory: /tmp/pip-ephem-wheel-cache-1klk2fsu/wheels/80/a8/35/f3e2a85eff1b8cb7e5e54cbd69a209356cf2153f9a4ee67904
Successfully built tf-utils


In [5]:
import tf_utils.data as ds 
import tf_utils.visualize as vz
import tf_utils.transform as tfm

In [6]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


# New Section

In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [8]:
dirlist = lambda di: [os.path.join(di, file) for file in os.listdir(di) if 'part-' in file]
training_files = dirlist('/gdrive/MyDrive/wandb-dw/data/droughtwatch_data/train/')
val_files = dirlist('/gdrive/MyDrive/wandb-dw/data/droughtwatch_data/val/')



In [9]:
train_ds=ds.get_train_ds(training_files,batch_size=ds.batch_size,shuffle=True,distort=True,distort_fn=tfm.aug1)
#unlabelled_ds=ds.get_unlabelled_ds()
test_ds=ds.get_test_ds(val_files)

distorting...
Finished 'get_tf_dataset_2' in 3.7878 secs
Finished 'get_tf_dataset_in_batches' in 3.7888 secs
Finished 'get_train_ds' in 3.7891 secs
Finished 'get_tf_dataset_2' in 0.0804 secs
Finished 'get_tf_dataset_in_batches' in 0.0808 secs
Finished 'get_test_ds' in 0.0811 secs


In [10]:
'''
for idx, row in enumerate(train_ds):
    print (row)
    if idx > 1:
        break

'''        

'\nfor idx, row in enumerate(train_ds):\n    print (row)\n    if idx > 1:\n        break\n\n'

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D,  Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.optimizers import Adam


  

In [12]:
'''
def conv(inp,f=32,k=3):
  conv_layer=Conv2D(f,k,use_bias=False,padding='same')(inp)
  conv_layer=BatchNormalization()(conv_layer)
  conv_layer=Activation('relu')(conv_layer)
  return conv_layer
'''  

"\ndef conv(inp,f=32,k=3):\n  conv_layer=Conv2D(f,k,use_bias=False,padding='same')(inp)\n  conv_layer=BatchNormalization()(conv_layer)\n  conv_layer=Activation('relu')(conv_layer)\n  return conv_layer\n"

In [13]:
import math
WEIGHT_DECAY=1.25e-4
reg=tf.keras.regularizers.l2(WEIGHT_DECAY)
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

def conv(inp,f=32,k=3):
  conv_layer=Conv2D(f,k,use_bias=False,padding='same',kernel_initializer=init_pytorch, kernel_regularizer=reg)(inp)
  conv_layer=BatchNormalization(momentum=0.9, epsilon=1e-5)(conv_layer)
  conv_layer=Activation('relu')(conv_layer)
  return conv_layer
def resBlk(inp,f=32,k=3,residual=True) :
  res1=conv(inp,f,k)
  res1=MaxPooling2D(pool_size=(2,2))(res1)
  if residual:
    res2=conv(res1,f,k)
    res3=conv(res2,f,k)
    return res1+res3
  else:
    return res1  

In [16]:
def build_model():
  f=64
  inp=Input(shape=(65,65,6))
  layer1=conv(inp,f,3)
  res1=resBlk(layer1,f*3,3)
  
  res2=resBlk(res1,f*6,3,False)
  
  res3=resBlk(res2,f*12,3)

  #res4=resBlk(res3,f*8,3,False)

  #res5=resBlk(res4,f*8,3)
  
  
  layer2=tf.keras.layers.GlobalMaxPooling2D()(res3)
  layer3=tf.keras.layers.Dense(4, kernel_initializer=init_pytorch, use_bias=False,kernel_regularizer=reg)(layer2)
  layer4=Lambda(lambda x: x*0.125)(layer3)
  out=Activation('softmax')(layer4)
  model=tf.keras.models.Model(inputs=[inp],outputs=[out])
  model.summary()
  return model 

In [17]:
model=build_model()
opt=SGD(lr=0.0001,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=['accuracy']
              )

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 65, 65, 6)]  0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 65, 65, 64)   3456        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 65, 65, 64)   256         conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 65, 65, 64)   0           batch_normalization_8[0][0]      
______________________________________________________________________________________________

In [18]:
class Log2wandb_Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    wandb.log({
        "Epoch": epoch,
        "Train Loss": logs["loss"],
        "Train Acc": logs["accuracy"],
        "Val Loss": logs["val_loss"],
        "val_acc": logs["val_accuracy"],
        "LR":model.optimizer.lr.numpy()
        
        })
        

In [19]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule():
    
    def schedule(epoch):
      
      lr=np.interp([epoch],[0, EPOCHS//5,EPOCHS//2,EPOCHS], [0.0001, 0.1, 0.006,0.001])[0]
      print('epoch ', epoch+1, ': setting learning rate to ',lr)
      return lr
    
    return LearningRateScheduler(schedule)

lr_sched = lr_schedule()

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_cpt=ModelCheckpoint(filepath='/gdrive/MyDrive/wandb-dw/best_model2.h5', 
                          verbose=1, save_best_only=True,monitor='val_accuracy',mode='auto')

In [21]:
import numpy as np
batch_size=128
EPOCHS=20
callback_list=[Log2wandb_Callback(),lr_sched,model_cpt]
model.fit(train_ds,epochs=EPOCHS, steps_per_epoch=np.ceil(86317/batch_size), 
          validation_data=test_ds, validation_steps=np.ceil(10778/batch_size),
          callbacks=callback_list,
          verbose=1)

Epoch 1/20
epoch  1 : setting learning rate to  0.0001
675/675 [==============================] - 461s 647ms/step - loss: 1.2179 - accuracy: 0.5767 - val_loss: 1.3341 - val_accuracy: 0.5953

Epoch 00001: val_accuracy improved from -inf to 0.59531, saving model to /gdrive/MyDrive/wandb-dw/best_model2.h5
Epoch 2/20
epoch  2 : setting learning rate to  0.025075
675/675 [==============================] - 78s 116ms/step - loss: 1.0850 - accuracy: 0.6265 - val_loss: 1.1056 - val_accuracy: 0.6441

Epoch 00002: val_accuracy improved from 0.59531 to 0.64412, saving model to /gdrive/MyDrive/wandb-dw/best_model2.h5
Epoch 3/20
epoch  3 : setting learning rate to  0.050050000000000004
675/675 [==============================] - 78s 116ms/step - loss: 0.9779 - accuracy: 0.6698 - val_loss: 1.1051 - val_accuracy: 0.5861

Epoch 00003: val_accuracy did not improve from 0.64412
Epoch 4/20
epoch  4 : setting learning rate to  0.07502500000000001
675/675 [==============================] - 78s 116ms/step - l

In [25]:
def lr_schedule():
    
    def schedule(epoch):
      
      lr=np.interp([epoch],[20,25,EPOCHS], [0.001,0.005, 0.0001])[0]
      print('epoch ', epoch+1, ': setting learning rate to ',lr)
      return lr
    
    return LearningRateScheduler(schedule)

lr_sched = lr_schedule()

In [27]:
cust = {'init_pytorch': init_pytorch}


In [28]:
model=load_model('/gdrive/MyDrive/wandb-dw/best_model2.h5',cust)

In [29]:
opt=SGD(lr=0.001,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=['accuracy']
              )

In [30]:
#train_ds=ds.get_train_ds(training_files,batch_size=ds.batch_size,shuffle=True,distort=True,distort_fn=tfm.aug2)

In [31]:
import numpy as np
batch_size=128
EPOCHS=40
callback_list=[Log2wandb_Callback(),lr_sched,model_cpt]
model.fit(train_ds,epochs=EPOCHS, steps_per_epoch=np.ceil(86317/batch_size), initial_epoch=20,
          validation_data=test_ds, validation_steps=np.ceil(10778/batch_size),
          callbacks=callback_list,
          verbose=1)

Epoch 21/40
epoch  21 : setting learning rate to  0.001
675/675 [==============================] - 80s 117ms/step - loss: 0.5768 - accuracy: 0.8127 - val_loss: 0.6630 - val_accuracy: 0.7745

Epoch 00021: val_accuracy did not improve from 0.77583
Epoch 22/40
epoch  22 : setting learning rate to  0.0018
675/675 [==============================] - 78s 116ms/step - loss: 0.5694 - accuracy: 0.8154 - val_loss: 0.6641 - val_accuracy: 0.7785

Epoch 00022: val_accuracy improved from 0.77583 to 0.77849, saving model to /gdrive/MyDrive/wandb-dw/best_model2.h5
Epoch 23/40
epoch  23 : setting learning rate to  0.0026
675/675 [==============================] - 78s 116ms/step - loss: 0.5693 - accuracy: 0.8144 - val_loss: 0.6639 - val_accuracy: 0.7753

Epoch 00023: val_accuracy did not improve from 0.77849
Epoch 24/40
epoch  24 : setting learning rate to  0.0034000000000000002
675/675 [==============================] - 78s 116ms/step - loss: 0.5688 - accuracy: 0.8136 - val_loss: 0.6706 - val_accuracy: 

In [37]:
train_ds=ds.get_train_ds(training_files,batch_size=ds.batch_size,shuffle=True,distort=True,distort_fn=tfm.aug1)

distorting...
Finished 'get_tf_dataset_2' in 0.2070 secs
Finished 'get_tf_dataset_in_batches' in 0.2073 secs
Finished 'get_train_ds' in 0.2077 secs


In [41]:
model=load_model('/gdrive/MyDrive/wandb-dw/best_model2.h5',cust)

In [48]:
opt=SGD(lr=0.00042,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=['accuracy']
              )

In [50]:
EPOCHS=41
callback_list=[Log2wandb_Callback(),model_cpt]
model.fit(train_ds,epochs=EPOCHS, steps_per_epoch=np.ceil(86317/batch_size), initial_epoch=40,
          validation_data=test_ds, validation_steps=np.ceil(10778/batch_size),
          callbacks=callback_list,
          verbose=1)

Epoch 41/41
675/675 [==============================] - 78s 116ms/step - loss: 0.4693 - accuracy: 0.8524 - val_loss: 0.6566 - val_accuracy: 0.7803

Epoch 00041: val_accuracy did not improve from 0.78171


In [51]:
run.finish()

Epoch,40
Train Loss,0.46932
Train Acc,0.85243
Val Loss,0.65664
val_acc,0.78033
LR,0.00042
_runtime,5345
_timestamp,1619021234
_step,48


Epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████████
Train Loss,█▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Acc,▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
Val Loss,█▆▆▄▄▃▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▁▅▅▅▆▇▇███▇███████████████████████████
LR,▁▃▅▆█▆▅▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
_timestamp,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
